In [1]:
import json
from colorama import Fore, Style
from utils import rewrite_DiffMarks, replace_code_blocks_with_classification
from prompt_role import gpt4_lineRole, gpt4_tokenRole, cot_lineWithRole, cot_tokenWithRole, cot_lineNoRole, cot_tokenNoRole, cot_presuf
import sys
from openai import OpenAI
from utils import delete_threePoint, extract_res_fromChatGPT, extract_res_fromChatGPT2
sys.argv=['']
del sys

In [2]:
"""配置API key(若是中转 则需要配置中转链接)"""
client = OpenAI(
    base_url="https://api.xiaoai.plus/v1", # Tao 2024年4月27日18:54:47
    api_key="sk-BHV2QfUc2EbeyRnK4eEd3b90BbC341E4B2881086891b0864" 
)

"""ChatGPT(GPT-3.5) 一次对话"""
def ChatGPT_35Code(question, number, role, truth_resolution):

    all_responses = [] 
    print(question)
    print(f"\n<Truth Resolution>:-------->\n{truth_resolution}\n</Truth Resolution>-------->")
    for i in range(number):
        print(f"\n---------------ChatGPT的第{i + 1}个Resolution：---------------")

        response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106", # gpt-3.5-turbo-1106 # gpt-3.5-turbo
            messages=[
                {
                "role": "system", "content": role # 指定角色(Java高级开发工程师)
                },
                {
                "role": "user", "content": question
                }
            ], 
            max_tokens = 2048, # This model's maximum context length is 4097 tokens.
            temperature = 0, # 0-2之间，越大越随机，越小越确定 0.01
        )
        
        current_response = response.choices[0].message.content

        # 这里是COT的处理
        print(current_response)
        only_code = delete_threePoint(extract_res_fromChatGPT(current_response))
        if only_code == "null" or only_code == "":
            only_code = delete_threePoint(extract_res_fromChatGPT2(current_response))
            all_responses.append(only_code)
        else:
            all_responses.append(only_code)

    return all_responses

In [ ]:
"""此方法是遍历100个示例JSON进行获取ChatGPT答案并保存 使用chain of thought的Prompt方式"""
def run_100_COT(input_100JsonPath, output_100JsonPath):

    try:
        with open(input_100JsonPath, 'r', encoding='utf-8') as input_file:
            json_data = json.load(input_file)

        for i in range(len(json_data)):
            if i + 1 < 216: # 从异常发生点接着运行
                continue

            item = json_data[i]
            key_context = item.get('embedding_key_context', [])
            truth_resolution = item.get('res_region')

            keyContext_set = set()
            for key_context_item in key_context: # embedding 上下文
                keyContext_set.add(key_context_item.get('a_keyContext'))
                keyContext_set.add(key_context_item.get('o_keyContext'))
                keyContext_set.add(key_context_item.get('b_keyContext'))
          
            key_context_all = ""
            for index, element in enumerate(keyContext_set):
                if element.replace(" ", "") != "":
                    key_context_all += f"上下文代码行{index + 1}:{element}\n"

            line_level_conflict = "\n<<<<<<<\n" + item['a_contents'] + "|||||||\n" + item['o_contents'] + "=======\n" + item['b_contents'] + ">>>>>>>\n"
            token_level_conflict = item['token_level_result']
            token_level_conflict = rewrite_DiffMarks(token_level_conflict) # rewrite diff marks

            print(f"\n<--------------------------------------------示例{i + 1}(line_withContext)：-------------------------------------------->")
            item['line_withContext_answer'] = ChatGPT_35Code(question="<context>\n" + key_context_all + "</context>\n\n<conflictBlock>" + line_level_conflict + "</conflictBlock>", 
                                                            number=5, 
                                                            role=cot_lineWithRole, truth_resolution=truth_resolution)
            # time.sleep(20)

            print(f"\n<--------------------------------------------示例{i + 1}(token_withContext)：-------------------------------------------->")
            item['token_withContext_answer'] = ChatGPT_35Code(question="<context>\n" + key_context_all + "</context>\n\n<conflictBlock>\n" + token_level_conflict + "</conflictBlock>", 
                                                            number=5, 
                                                            role=cot_tokenWithRole, truth_resolution=truth_resolution)
            # time.sleep(20)

    except Exception as e:
        print(f"{Fore.RED}Error----:An error occurred: {e}{Style.RESET_ALL}")
    finally:
        # 发生异常时先立即保存已运行数据
        with open(output_100JsonPath, 'w', encoding='utf-8') as output_file:
            json.dump(json_data, output_file, indent=2)
        print(f"{Fore.GREEN}Warnning----:ChatGPT已运行数据已保存！异常发生点：第{i + 1}示例{Style.RESET_ALL}")
        output_file.close()

"""调用使用ChatGPT运行100示例方法""" # 要使用COT先去更换api中的答案抽取！！！
run_100_COT(input_100JsonPath = "G:/now/2024merge/mergeMinePython/output/50Repo_Output/50_500v1_embeddingNew.json", # 处理异常的时候这里也要改成相同的文件名
            output_100JsonPath= "G:/now/2024merge/mergeMinePython/output/50Repo_Output/50_500v1_embeddingNew.json") 
